# Week 1 Prototype: Attack-Target Network

This notebook implements **Week 1** from `analysis/data/ATTACK_TARGET_NETWORK_PROJECT_PROPOSAL.md`:
1. Build harmonized text table schema
2. Implement text selection + cleaning
3. Run NER on a 1-5% sample
4. Draft alias-mapping workflow

Outputs are written to `analysis/outputs/week1/`.

## Notebook Scope and Constraints

- This is a **prototype pipeline** for development and QA, not final production extraction.
- Sampling is deterministic and configurable (default 2% with row caps).
- NER is limited to `PERSON`, `ORG`, `GPE` targets for the attack-target project.

In [1]:
from pathlib import Path
import re
import hashlib
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

In [2]:
# Project paths
PROJECT_ROOT = Path.cwd().resolve()

# Resolve robustly for both notebook-dir and repo-root execution
if (PROJECT_ROOT / 'data').exists() and (PROJECT_ROOT / 'notebooks').exists():
    # running from analysis/
    ANALYSIS_ROOT = PROJECT_ROOT
elif PROJECT_ROOT.name == 'notebooks' and (PROJECT_ROOT.parent / 'data').exists():
    # running from analysis/notebooks/
    ANALYSIS_ROOT = PROJECT_ROOT.parent
elif (PROJECT_ROOT / 'analysis' / 'data').exists():
    # running from repository root
    ANALYSIS_ROOT = PROJECT_ROOT / 'analysis'
else:
    raise RuntimeError('Could not resolve analysis root. Run from analysis/, analysis/notebooks/, or repo root.')

DATA_ROOT = ANALYSIS_ROOT / 'data'
OUT_ROOT = ANALYSIS_ROOT / 'outputs' / 'week1'
OUT_ROOT.mkdir(parents=True, exist_ok=True)

FILES = {
    'google': DATA_ROOT / 'raw/digital/2024/google/google2024_set1_20250715.csv.gz',
    'meta': DATA_ROOT / 'raw/digital/2024/meta/meta2024_set1_20250714.csv.gz',
    'tv': DATA_ROOT / 'raw/tv/2024/issues_by_creative/Ads2024_IssuesbyCreative_090124-110624_HSE_AI_013026.csv',
}

for k, p in FILES.items():
    print(f'{k:>6}:', p, 'exists=', p.exists())

print('Analysis root:', ANALYSIS_ROOT)
print('Output dir:', OUT_ROOT)

google: /Users/jeremyzay/Desktop/delta_lab/analysis/data/raw/digital/2024/google/google2024_set1_20250715.csv.gz exists= True
  meta: /Users/jeremyzay/Desktop/delta_lab/analysis/data/raw/digital/2024/meta/meta2024_set1_20250714.csv.gz exists= True
    tv: /Users/jeremyzay/Desktop/delta_lab/analysis/data/raw/tv/2024/issues_by_creative/Ads2024_IssuesbyCreative_090124-110624_HSE_AI_013026.csv exists= True
Analysis root: /Users/jeremyzay/Desktop/delta_lab/analysis
Output dir: /Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1


## Configuration

Adjust these defaults for speed vs coverage. Current settings target Week 1 prototype scale.

In [3]:
CFG = {
    'sample_frac': 0.02,            # 1-5% target; set anywhere in [0.01, 0.05]
    'chunk_size': 100_000,
    'max_rows_per_platform': 35_000,
    'min_text_chars': 20,
    'seed': 42,
}
CFG

{'sample_frac': 0.02,
 'chunk_size': 100000,
 'max_rows_per_platform': 35000,
 'min_text_chars': 20,
 'seed': 42}

## Harmonization Rules (Locked v1.1)

Implements the proposal decision rules:
- canonical party mapping (`party_std`)
- tone fallback per platform (`tone_std`)
- unified schema fields for downstream graph work

In [4]:
MISSING_TOKENS = {'', 'na', 'n/a', 'null', 'none', '\\n', '\\N'}

def is_missing(x):
    if x is None:
        return True
    s = str(x).strip()
    return s.lower() in MISSING_TOKENS

def clean_text(x):
    if is_missing(x):
        return ''
    s = str(x)
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'www\.\S+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = s.strip()
    return s

def map_party(raw):
    if is_missing(raw):
        return 'UNKNOWN', 'low'
    s = str(raw).strip().lower()
    if 'dem' in s:
        return 'DEM', 'high'
    if 'rep' in s or 'gop' in s:
        return 'REP', 'high'
    if 'ind' in s:
        return 'IND', 'high'
    if 'non' in s and 'part' in s:
        return 'NONPARTISAN', 'medium'
    if s in {'unknown', 'other'}:
        return 'OTHER', 'medium'
    return 'OTHER', 'low'

def map_tone(raw):
    if is_missing(raw):
        return 'UNKNOWN'
    s = str(raw).strip().lower()
    if 'contrast' in s:
        return 'CONTRAST'
    if 'neg' in s:
        return 'NEGATIVE'
    if 'pos' in s:
        return 'POSITIVE'
    if 'mix' in s:
        return 'MIXED'
    return 'UNKNOWN'

def hash_select(ad_id, platform, frac=0.02):
    # Deterministic sample filter by stable hash
    key = f'{platform}|{ad_id}'
    h = hashlib.sha1(key.encode('utf-8')).hexdigest()
    # map first 8 hex chars to [0,1)
    v = int(h[:8], 16) / 16**8
    return v < frac

In [5]:
def build_google(chunk):
    # tone fallback: ad_tone -> ad_tone_constructed -> UNKNOWN
    tone_raw = chunk['ad_tone'].where(~chunk['ad_tone'].isna(), chunk['ad_tone_constructed'])

    # text preference: asr_text -> ocr_text
    text_raw = chunk['asr_text'].where(~chunk['asr_text'].isna(), chunk['ocr_text'])

    out = pd.DataFrame({
        'platform': 'google',
        'ad_id': chunk['ad_id'].astype(str),
        'sponsor_name': chunk['advertiser_name'].astype(str),
        'party_raw': chunk.get('party_all', pd.Series(index=chunk.index, dtype='object')),
        'party_source': 'party_all',
        'office_raw': chunk.get('office_corrected', pd.Series(index=chunk.index, dtype='object')),
        'tone_raw': tone_raw,
        'tone_source': np.where(chunk['ad_tone'].notna(), 'ad_tone', np.where(chunk['ad_tone_constructed'].notna(), 'ad_tone_constructed', 'none')),
        'issue_context': chunk.get('race_of_focus', pd.Series(index=chunk.index, dtype='object')),
        'text_main': text_raw,
        'date': chunk.get('first_served_timestamp', chunk.get('date_range_start', pd.Series(index=chunk.index, dtype='object'))),
        'spend_proxy': chunk.get('spend', (chunk.get('spend_range_min_usd', 0).fillna(0) + chunk.get('spend_range_max_usd', 0).fillna(0)) / 2),
    })
    return out

def build_meta(chunk):
    # tone fallback: ad_tone -> ad_tone_constructed -> UNKNOWN
    tone_raw = chunk['ad_tone'].where(~chunk['ad_tone'].isna(), chunk['ad_tone_constructed'])

    # text preference: ad_creative_body (+link context) -> ocr_text
    body = chunk['ad_creative_body'].fillna('')
    link_title = chunk['ad_creative_link_title'].fillna('')
    link_desc = chunk['ad_creative_link_description'].fillna('')
    combined = (body + ' ' + link_title + ' ' + link_desc).str.strip()
    text_raw = combined.where(combined.str.len() > 0, chunk['ocr_text'])

    party_raw = chunk.get('party_group', pd.Series(index=chunk.index, dtype='object'))
    party_source = np.where(party_raw.notna(), 'party_group', 'unknown')

    out = pd.DataFrame({
        'platform': 'meta',
        'ad_id': chunk['ad_id'].astype(str),
        'sponsor_name': chunk['page_name'].astype(str),
        'party_raw': party_raw,
        'party_source': party_source,
        'office_raw': chunk.get('office_corrected', pd.Series(index=chunk.index, dtype='object')),
        'tone_raw': tone_raw,
        'tone_source': np.where(chunk['ad_tone'].notna(), 'ad_tone', np.where(chunk['ad_tone_constructed'].notna(), 'ad_tone_constructed', 'none')),
        'issue_context': chunk.get('race_of_focus', pd.Series(index=chunk.index, dtype='object')),
        'text_main': text_raw,
        'date': chunk.get('ad_delivery_start_time', chunk.get('first_day_active', pd.Series(index=chunk.index, dtype='object'))),
        'spend_proxy': chunk.get('spend', (chunk.get('spend_lower', 0).fillna(0) + chunk.get('spend_upper', 0).fillna(0)) / 2),
    })
    return out

def build_tv(chunk):
    # tone: tone -> UNKNOWN
    text_raw = chunk['transcript'].where(~chunk['transcript'].isna(), chunk['title'])

    issue_context = chunk['issue_1'].fillna('')
    issue_context = issue_context + np.where(chunk['issue_2'].fillna('').str.len() > 0, '|' + chunk['issue_2'].fillna(''), '')
    issue_context = issue_context + np.where(chunk['issue_3'].fillna('').str.len() > 0, '|' + chunk['issue_3'].fillna(''), '')

    out = pd.DataFrame({
        'platform': 'tv',
        'ad_id': chunk['uuid'].astype(str),
        'sponsor_name': chunk['advertiser'].astype(str),
        'party_raw': chunk.get('advertiser_party', pd.Series(index=chunk.index, dtype='object')),
        'party_source': 'advertiser_party',
        'office_raw': chunk.get('category', chunk.get('race_alt', pd.Series(index=chunk.index, dtype='object'))),
        'tone_raw': chunk.get('tone', pd.Series(index=chunk.index, dtype='object')),
        'tone_source': 'tone',
        'issue_context': issue_context,
        'text_main': text_raw,
        'date': chunk.get('airdate', pd.Series(index=chunk.index, dtype='object')),
        'spend_proxy': chunk.get('spent', pd.Series(index=chunk.index, dtype='object')),
    })
    return out

def finalize_harmonized(df):
    df = df.copy()
    df['text_main'] = df['text_main'].map(clean_text)
    df['text_len'] = df['text_main'].str.len()

    mapped = df['party_raw'].map(map_party)
    df['party_std'] = mapped.map(lambda x: x[0])
    df['party_confidence'] = mapped.map(lambda x: x[1])
    df['tone_std'] = df['tone_raw'].map(map_tone)

    # office standardization (lightweight v1)
    df['office_std'] = df['office_raw'].astype(str).str.upper().str.strip()
    df.loc[df['office_std'].isin(['', 'NAN', 'NONE']), 'office_std'] = 'UNKNOWN'

    # sample filter and quality filter
    df = df[df['text_len'] >= CFG['min_text_chars']].copy()
    return df

In [6]:
def sample_harmonized(platform, path):
    builder = {'google': build_google, 'meta': build_meta, 'tv': build_tv}[platform]
    out_parts = []
    rows_kept = 0

    reader = pd.read_csv(path, chunksize=CFG['chunk_size'], compression='gzip' if str(path).endswith('.gz') else None, low_memory=False)
    for i, chunk in enumerate(reader, start=1):
        h = builder(chunk)
        h = finalize_harmonized(h)
        mask = h['ad_id'].map(lambda x: hash_select(x, platform, CFG['sample_frac']))
        h = h[mask]

        if not h.empty:
            out_parts.append(h)
            rows_kept += len(h)

        if rows_kept >= CFG['max_rows_per_platform']:
            break

        if i % 5 == 0:
            print(f'[{platform}] chunks={i}, rows_kept={rows_kept:,}')

    if out_parts:
        out = pd.concat(out_parts, ignore_index=True).head(CFG['max_rows_per_platform'])
    else:
        out = pd.DataFrame()

    return out

sample_frames = []
for platform, path in FILES.items():
    print(f'Building sample for {platform}...')
    sdf = sample_harmonized(platform, path)
    print(f'  -> {len(sdf):,} rows')
    sample_frames.append(sdf)

harmonized_sample = pd.concat(sample_frames, ignore_index=True)
print('Total harmonized sample rows:', f"{len(harmonized_sample):,}")

harmonized_sample.head(3)

Building sample for google...
  -> 1,599 rows
Building sample for meta...
[meta] chunks=5, rows_kept=9,579
  -> 9,579 rows
Building sample for tv...
[tv] chunks=5, rows_kept=9,916
[tv] chunks=10, rows_kept=18,652
  -> 18,652 rows
Total harmonized sample rows: 29,830


,platform,ad_id,sponsor_name,party_raw,party_source,office_raw,tone_raw,tone_source,issue_context,text_main,date,spend_proxy,text_len,party_std,party_confidence,tone_std,office_std
0,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,party_all,NaN,Contrast,ad_tone,AZS0,any good cop can tell good from bad and truth ...,2024-08-26 14:58:00 UTC,1750.0,460,REP,high,CONTRAST,NaN
1,google,CR03113750924804227073,MONICA TRANEL FOR MONTANA,DEM,party_all,NaN,Contrast,ad_tone,MT01,we're about to go inside one of Ryan zin's rea...,2024-08-27 18:28:00 UTC,2250.0,206,DEM,high,CONTRAST,NaN
2,google,CR12862966046910316545,HOVDE FOR WISCONSIN,REP,party_all,NaN,Contrast,ad_tone,WIS0,I just don't feel safe anymore there's too muc...,2024-08-07 18:19:00 UTC,2250.0,229,REP,high,CONTRAST,NaN


In [7]:
# IO helper: parquet with fallback for pyarrow/pandas edge cases
def write_table_safe(df, out_base_path):
    """
    Try writing parquet first. If parquet fails (e.g., pyarrow compatibility issues),
    fallback to pickle and csv.gz. Returns a metadata dict.
    """
    out_base_path = Path(out_base_path)

    try:
        out_parquet = out_base_path.with_suffix('.parquet')
        df.to_parquet(out_parquet, index=False)
        return {
            'primary_path': str(out_parquet),
            'primary_format': 'parquet',
            'fallback_paths': []
        }
    except Exception as e:
        print('Parquet write failed, using fallback formats.')
        print('Error:', repr(e))

        out_pickle = out_base_path.with_suffix('.pkl')
        out_csv_gz = out_base_path.with_suffix('.csv.gz')
        df.to_pickle(out_pickle)
        df.to_csv(out_csv_gz, index=False, compression='gzip')

        return {
            'primary_path': str(out_pickle),
            'primary_format': 'pickle',
            'fallback_paths': [str(out_csv_gz)]
        }

In [8]:
# Persist harmonized Week 1 sample
harmonized_io = write_table_safe(harmonized_sample, OUT_ROOT / 'harmonized_sample_week1')
harmonized_path = Path(harmonized_io['primary_path'])
print('Harmonized output:', harmonized_io)

summary = (
    harmonized_sample
    .groupby('platform')
    .agg(
        rows=('ad_id', 'count'),
        avg_text_len=('text_len', 'mean'),
        unique_sponsors=('sponsor_name', 'nunique'),
        share_negative_or_contrast=('tone_std', lambda s: (s.isin(['NEGATIVE','CONTRAST'])).mean())
    )
    .reset_index()
)
summary

Harmonized output: {'primary_path': '/Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1/harmonized_sample_week1.parquet', 'primary_format': 'parquet', 'fallback_paths': []}


,platform,rows,avg_text_len,unique_sponsors,share_negative_or_contrast
0,google,1599,437.499687,454,0.233271
1,meta,9579,318.519052,2390,0.132269
2,tv,18652,454.857602,356,0.673547


## Step B: NER on 1-5% Sample

This cell loads spaCy and extracts candidate target entities (`PERSON`, `ORG`, `GPE`).
If no model is installed, run the install cell below once.

In [9]:
# Optional install (uncomment if needed)
# !pip install -q spacy
# !python -m spacy download en_core_web_sm

In [10]:
import spacy

MODEL_CANDIDATES = ['en_core_web_trf', 'en_core_web_lg', 'en_core_web_md', 'en_core_web_sm']

def load_spacy_model():
    last_err = None
    for m in MODEL_CANDIDATES:
        try:
            nlp = spacy.load(m)
            print('Loaded model:', m)
            return nlp, m
        except Exception as e:
            last_err = e
    raise RuntimeError(f'No spaCy English model found. Install one (e.g., en_core_web_sm). Last error: {last_err}')

nlp, nlp_model_name = load_spacy_model()
TARGET_LABELS = {'PERSON', 'ORG', 'GPE'}

Loaded model: en_core_web_sm


In [11]:
def extract_entities(df, batch_size=64):
    rows = []
    docs = nlp.pipe(df['text_main'].tolist(), batch_size=batch_size)

    for idx, doc in enumerate(docs):
        row = df.iloc[idx]
        for ent in doc.ents:
            if ent.label_ not in TARGET_LABELS:
                continue
            rows.append({
                'platform': row['platform'],
                'ad_id': row['ad_id'],
                'sponsor_name': row['sponsor_name'],
                'party_std': row['party_std'],
                'office_std': row['office_std'],
                'tone_std': row['tone_std'],
                'date': row['date'],
                'entity_text': ent.text,
                'entity_label': ent.label_,
                'start_char': ent.start_char,
                'end_char': ent.end_char,
                'context_window': doc[max(0, ent.start-8):min(len(doc), ent.end+8)].text,
            })

    return pd.DataFrame(rows)

mentions = extract_entities(harmonized_sample)
print('Mentions extracted:', f"{len(mentions):,}")
mentions.head(10)

Mentions extracted: 127,464


,platform,ad_id,sponsor_name,party_std,office_std,tone_std,date,entity_text,entity_label,start_char,end_char,context_window
0,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,NaN,CONTRAST,2024-08-26 14:58:00 UTC,Ruben,ORG,75,80,bad and truth from lies and Democrats like Rub...
1,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,NaN,CONTRAST,2024-08-26 14:58:00 UTC,Congress,ORG,163,171,talks stuff now but for four years in Congress...
2,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,NaN,CONTRAST,2024-08-26 14:58:00 UTC,camela Harris,PERSON,182,195,but for four years in Congress he backed camel...
3,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,NaN,CONTRAST,2024-08-26 14:58:00 UTC,Ruben,ORG,376,381,in tragic overdose deaths this crisis happened...
4,google,CR17105064975556149249,KARI LAKE FOR SENATE,REP,NaN,CONTRAST,2024-08-26 14:58:00 UTC,Carrie Lake,PERSON,430,441,n't let it continue on yours I'm Carrie Lake a...
5,google,CR03113750924804227073,MONICA TRANEL FOR MONTANA,DEM,NaN,CONTRAST,2024-08-27 18:28:00 UTC,Ryan,PERSON,32,36,we're about to go inside one of Ryan zin's rea...
6,google,CR03113750924804227073,MONICA TRANEL FOR MONTANA,DEM,NaN,CONTRAST,2024-08-27 18:28:00 UTC,Ryan,PERSON,137,141,"rent $166,000 a month for this place Ryan ziny..."
7,google,CR03113750924804227073,MONICA TRANEL FOR MONTANA,DEM,NaN,CONTRAST,2024-08-27 18:28:00 UTC,Monica,PERSON,174,180,Ryan ziny is a housing profiteer I'm Monica tr...
8,google,CR12862966046910316545,HOVDE FOR WISCONSIN,REP,NaN,CONTRAST,2024-08-07 18:19:00 UTC,Tammy Baldwin,PERSON,79,92,'s too much crime in our neighborhoods and Tam...
9,google,CR12862966046910316545,HOVDE FOR WISCONSIN,REP,NaN,CONTRAST,2024-08-07 18:19:00 UTC,Tammy Baldwin,PERSON,140,153,Baldwin supports lowering jail time for violen...


In [12]:
mentions_io = write_table_safe(mentions, OUT_ROOT / 'entity_mentions_week1')
mentions_path = Path(mentions_io['primary_path'])
print('Mentions output:', mentions_io)

entity_counts = (
    mentions.groupby(['entity_text', 'entity_label'], as_index=False)
    .size()
    .rename(columns={'size': 'mention_count'})
    .sort_values('mention_count', ascending=False)
)

entity_counts.head(25)

Mentions output: {'primary_path': '/Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1/entity_mentions_week1.parquet', 'primary_format': 'parquet', 'fallback_paths': []}


,entity_text,entity_label,mention_count
1686,Congress,ORG,6054
8663,Washington,GPE,2827
5266,Medicare,ORG,2066
4131,Kamala Harris,PERSON,1511
267,Alaska,GPE,1413
357,America,GPE,1191
2239,Donald Trump,PERSON,1030
4003,Josh Riley,PERSON,1022
8199,Trump,PERSON,932
5883,New York,GPE,930


## Step C: Draft Alias Mapping Workflow

Creates machine-suggested canonical names for high-frequency entities.
You should manually review and edit before using in final graph construction.

In [13]:
def normalize_entity_seed(s):
    s = str(s).strip().lower()
    s = re.sub(r'^(mr|mrs|ms|dr|president|senator|rep|representative|gov|governor)\.??\s+', '', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

alias_candidates = entity_counts.copy()
alias_candidates['canonical_suggested'] = alias_candidates['entity_text'].map(normalize_entity_seed)
alias_candidates['needs_review'] = True
alias_candidates['review_notes'] = ''

# Prioritize by frequency for manual review
alias_candidates = alias_candidates.sort_values(['mention_count', 'entity_label'], ascending=[False, True])

# Keep a practical review file for Week 1
alias_review = alias_candidates.head(2000).copy()
alias_review_path = OUT_ROOT / 'entity_alias_candidates_week1.csv'
alias_review.to_csv(alias_review_path, index=False)

alias_review.head(20)

,entity_text,entity_label,mention_count,canonical_suggested,needs_review,review_notes
1686,Congress,ORG,6054,congress,True,
8663,Washington,GPE,2827,washington,True,
5266,Medicare,ORG,2066,medicare,True,
4131,Kamala Harris,PERSON,1511,kamala harris,True,
267,Alaska,GPE,1413,alaska,True,
357,America,GPE,1191,america,True,
2239,Donald Trump,PERSON,1030,donald trump,True,
4003,Josh Riley,PERSON,1022,josh riley,True,
8199,Trump,PERSON,932,trump,True,
5883,New York,GPE,930,new york,True,


In [14]:
# QA / progress artifacts
qa = {
    'nlp_model': nlp_model_name,
    'sample_frac': CFG['sample_frac'],
    'rows_harmonized': int(len(harmonized_sample)),
    'rows_by_platform': harmonized_sample['platform'].value_counts().to_dict(),
    'mentions_extracted': int(len(mentions)),
    'unique_entities': int(mentions['entity_text'].nunique()) if len(mentions) else 0,
    'outputs': {
        'harmonized_sample': harmonized_io,
        'entity_mentions': mentions_io,
        'alias_candidates': str(alias_review_path),
    },
}
qa

{'nlp_model': 'en_core_web_sm',
 'sample_frac': 0.02,
 'rows_harmonized': 29830,
 'rows_by_platform': {'tv': 18652, 'meta': 9579, 'google': 1599},
 'mentions_extracted': 127464,
 'unique_entities': 10367,
 'outputs': {'harmonized_sample': {'primary_path': '/Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1/harmonized_sample_week1.parquet',
   'primary_format': 'parquet',
   'fallback_paths': []},
  'entity_mentions': {'primary_path': '/Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1/entity_mentions_week1.parquet',
   'primary_format': 'parquet',
   'fallback_paths': []},
  'alias_candidates': '/Users/jeremyzay/Desktop/delta_lab/analysis/outputs/week1/entity_alias_candidates_week1.csv'}}

## Week 1 Done Checklist

- [x] Harmonized text schema created
- [x] Text selection and cleaning implemented
- [x] NER run on sampled data
- [x] Alias mapping candidate workflow drafted

### Suggested Next Step (Week 2)
- Manually review `entity_alias_candidates_week1.csv` and lock canonical entity map before full extraction.